# Data Cleaning
This notebook will go trough the cleaning process of the file The_Office.csv, extracted with The_Office_Scraper.py.

The main objective is make sure the character names are correct, without typos, mismatches and errors in general.

## Prepare Enviorment

The libraries required to run this notebook are Pandas, Numpy, VaderSentiment, and NLTK.

* this notebook will also make use of requests, json, and re (regular expressions), those are standard libraries in python 3.

In [1]:
# install required libraries
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install vaderSentiment
!{sys.executable} -m pip install nltk

In [2]:
import re
import json
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

root_path = ''
df = pd.read_csv(root_path+'the_office.csv', sep=';', encoding='utf-16')
df.columns = ['id','char','text','ep']
df.head()

,id,char,text,ep
0,0,As you may be aware our board is experiencing ...,https://www.facebook.com/foreverdreaming.org/,Board Updates: Please Read 8/26/19
1,1,If you would like to help edit any of our tran...,),Board Updates: Please Read 8/26/19
2,2,Michael,All right Jim. Your quarterlies look very goo...,01x01 - Pilot
3,3,Jim,"Oh, I told you. I couldn't close it. So...",01x01 - Pilot
4,4,Michael,So you've come to the master for guidance? Is...,01x01 - Pilot


## Grouped Characters
Some dialogs had more than one characther talking the same thing at the same time, those were found in different ways like listed bellow.  
  
        char X and char Y;  
        char X & char Y;  
        char X, char Y and char Z;  
        char X, Y, Z;  
        char X/ char Y;  
        
The following script will break up those records in separate characters.

In [3]:
n_id = []
n_char = []
n_text = []
n_ep = []

for i, row in df.iterrows():
    # trim
    name = re.sub("[\[].*?[\]]", "", row.char).strip()
    name = name.replace('"','').replace(' together','')
    #separate characters
    #and
    if ' & ' in name:
        name = name.replace(' & ', ' and ')
    if ' and ' in name:
        temp = name.split(' and ')
        name = temp[0]
        if name.endswith(','):
            name = name[:-1]
        
        n_id.append(row.id)
        n_char.append(temp[1])
        n_text.append(row.text)
        n_ep.append(row.ep)
    #comma
    if ',' in name:
        names = name.split(', ')
        name = names[0]
        for value in names[1:]:
            n_id.append(row.id)
            n_char.append(value)
            n_text.append(row.text)
            n_ep.append(row.ep)
    #slash
    if '/' in name and name != 'DunMiff/sys':
        names = name.split('/')
        name = names[0]
        for value in names[1:]:
            n_id.append(row.id)
            n_char.append(value)
            n_text.append(row.text)
            n_ep.append(row.ep)

    df.at[i,'char'] = name

In [4]:
df_add = pd.DataFrame(n_id)
df_add.columns = ['id']
df_add['char'] = n_char
df_add['text'] = n_text
df_add['ep'] = n_ep

df = df.append(df_add, sort=True)

In [5]:
df.reset_index(inplace = True)
df.tail()

,index,char,ep,id,text
60314,177,Vance Refrigeration,09x20 - Paper Airplane,59044,You can do it baby!
60315,178,Oscar,09x22/23 - A.A.R.M,59615,"Kevin, what?"
60316,179,Dwight,09x22/23 - A.A.R.M,59694,Sea horse.
60317,180,Dwight,09x22/23 - A.A.R.M,59696,Say that? Uncanny.
60318,181,Oscar,09x22/23 - A.A.R.M,59817,Shhh!


## Clean Names
The characters were analysed in groups and them individually to make sure they were correctly recorded, three blocks of code that can be found at the end of this notebook were used to analyse the characters.  
This website was used for research about the characters: https://theoffice.fandom.com/wiki/Main_Page
  
The result of this research was a .json file containing a dictionary with the mismatching values and the correct values.
  
Besides the corrections.json, some characters have similar names so the episode name was considered for separate them.

In [6]:
file = open('corrections.json')
corrections_json = file.read()
corrections = json.loads(corrections_json)

In [7]:
for i, row in df.iterrows():
    name=str(row.char)
    # rename
    if name in corrections:
        name = corrections[name]
    #roberts
    elif name == 'Robert':
        if str(row.ep) in ['07x11/12 - Classy Christmas (Parts 1&2)', '09x16 - Moving On']:
            name = 'Robert Lipton'
        else:
            name = 'Robert California'
    #davids
    elif name == 'David':
        if row.ep is '07x14 - The Seminar':
            name = 'David Brent'
        else:
            name = 'David Wallace'
    #Samuel and Samuel L. Chang(Dwight)
    elif name=='Samuel' and str(row.ep) is '07x17 - Threat Level Midnight':
        name = 'Dwight'
    
    elif name=='Nick' and str(row.ep) is '04x07/08 - Money (Parts 1&2)':
        name = 'Nick Figaro'
    
    df.at[i,'name'] = str(name)

In [8]:
df.tail()

,index,char,ep,id,text,name
60314,177,Vance Refrigeration,09x20 - Paper Airplane,59044,You can do it baby!,Vance Refrigeration
60315,178,Oscar,09x22/23 - A.A.R.M,59615,"Kevin, what?",Oscar
60316,179,Dwight,09x22/23 - A.A.R.M,59694,Sea horse.,Dwight
60317,180,Dwight,09x22/23 - A.A.R.M,59696,Say that? Uncanny.,Dwight
60318,181,Oscar,09x22/23 - A.A.R.M,59817,Shhh!,Oscar


## Remove Garbage
As much as the scrapper avoided collecting unrelated data, some couldn't be identified at that part of the process so we need to remove them.

In [9]:
remove_ep = ['01x99 - Deleted Scenes from Season 1',
             '02x99 - Deleted Scenes from Season 2',
             '03x99 - Deleted Scenes from Season 3',
             '04x99 - Deleted Scenes from Season 4',
             '05x99 - Deleted Scenes from Season 5',
             '08x99 - Deleted Scenes from Season 8',
             'Board Updates: Please Read 8/26/19',
             '06x00 - Subtle Sexuality Webisodes 1-3',
             '07x00 - The 3rd Floor Webisodes 1-3',
             '05x30 - Gag Reel',
             "05x00 - Kevin's Loan Webisodes 1-4",
             '03x00 - The Accountants Webisodes 1-10']
# values I couldn't identify or classify
remove_c = ['8', "I'm gonna try to get to bed by 8", 'Boom Box',
            "Vance Refrigeration", 'Both', 'Song', 'song', "Oscar's Computer",
            "Erin's Cell Phone", "Ryan's Voicemail", "Jim's voicemail",
            'Automated phone voice', 'Voice on CD player', 'Robotic Voice',
            'Voicemail', 'GPS', 'Offscreen', 'Computron','DunMiff/sys',
            'Off-camera', 'Video']
# drop above listed records
df = df.drop(df[df['ep'].isin(remove_ep)].index)
df = df.drop(df[df['char'].isin(remove_c)].index)

In [10]:
df.tail()

,index,char,ep,id,text,name
60313,176,Oscar,09x18 - Promos,58614,Hi honey!,Oscar
60315,178,Oscar,09x22/23 - A.A.R.M,59615,"Kevin, what?",Oscar
60316,179,Dwight,09x22/23 - A.A.R.M,59694,Sea horse.,Dwight
60317,180,Dwight,09x22/23 - A.A.R.M,59696,Say that? Uncanny.,Dwight
60318,181,Oscar,09x22/23 - A.A.R.M,59817,Shhh!,Oscar


## Trim texts

In [11]:
clean_txt=[]
for i, row in df.iterrows():
    txt = str(row.text)
    if '  ' in txt:
        txt = txt.replace('  ',' ').strip()
    # unmask some words
    if 'v*g1n*' in txt:
        txt = txt.replace('v*g1n*', 'vagina')
    if 's*x' in txt:
        txt = txt.replace('s*x', 'sex')
    if 'f***ing' in txt:
        txt = txt.replace('f***ing', 'fucking')    
    
    clean_txt.append(txt)
    
df['text'] = clean_txt

In [12]:
df.sort_values(by='id', inplace = True)
df.tail()

,index,char,ep,id,text,name
60132,60132,Creed,09x24/25 - Finale,60547,It all seems so very arbitrary. I applied for ...,Creed
60133,60133,Meredith,09x24/25 - Finale,60548,I just feel lucky that I got a chance to shar...,Meredith
60134,60134,Phyllis,09x24/25 - Finale,60549,I'm happy that this was all filmed so I can r...,Phyllis
60135,60135,Jim,09x24/25 - Finale,60550,I sold paper at this company for 12 years. My...,Jim
60136,60136,Pam,09x24/25 - Finale,60551,I thought it was weird when you picked us to ...,Pam


## Separate Episode Field
The episode name contains the episode number, the season number and the actual episode name. the bellow script will split them in three different fields

In [13]:
ep_number=[]
ep_name=[]
season=[]
for i, row in df.iterrows():
    temp = row.ep.split(' - ')
    ep_name.append(temp[-1])
    temp = temp[0].split('x')
    ep_number.append(temp[-1])
    season.append(temp[0])

df['episode_name']=ep_name
df['episode_number']=ep_number
df['season']=season

## Clean episode names
  
### Separating episodes
Some episodes were grouped in parts one and two, to separate them I had to watch the beggining of the episodes listed as 'part 2', taking note of the character who said the first dialog and the content of this dialog. 
Those were then used to separate the episodes where everything bellow the id of those dialogs are 'part 1' and everything above it is 'part 2'

### Correcting names
Some of the episodes names were also corrected to match the names on the IMDB dataset.        

In [14]:
for i, row in df.iterrows():
    if row.episode_name == 'Niagara (Parts 1&2)':
        if row.id < 32435:
            df.at[i,'episode_name'] = 'Niagara: Part 1'
            df.at[i,'episode_number'] = '04'
        else:
            df.at[i,'episode_name'] = 'Niagara: Part 2'
            df.at[i,'episode_number'] = '05'
    elif row.episode_name == 'The Delivery (Parts 1&2)':
        if row.id < 36124:
            df.at[i,'episode_name'] = 'The Delivery: Part 1'
            df.at[i,'episode_number'] = '17'
        else:
            df.at[i,'episode_name'] = 'The Delivery: Part 2'
            df.at[i,'episode_number'] = '18'
            
    # Correct some names
    elif row.episode_name == 'Lecture Circuit (Part 1)':
        df.at[i,'episode_name'] = 'Lecture Circuit: Part 1'
        
    elif row.episode_name == 'Lecture Circuit (Part 2)':
        df.at[i,'episode_name'] = 'Lecture Circuit: Part 2'
        
    elif row.episode_name == 'Boys & Girls':
        df.at[i,'episode_name'] = 'Boys and Girls'
    
    elif row.episode_name == 'A.A.R.M':
        df.at[i,'episode_name'] = 'A.A.R.M.'
        
    elif row.episode_name == 's*x Ed':
        df.at[i,'episode_name'] = 'Sex ED'
        
    elif row.episode_name == 'The Manager and the Salesman':
        df.at[i,'episode_name'] = 'Manager and Salesman'
    
    elif row.episode_name in ['The Michael Scott Paper Company', 'The New Guys']:
        df.at[i,'episode_name'] = row.episode_name[4:]    
        
    elif ' (Parts 1&2)' in row.episode_name:
        df.at[i,'episode_name'] = row.episode_name.replace(' (Parts 1&2)','')

## Last details
Build a dataframe to be saved, remove characters identified as random people and add an id field to uniquely identify episodes and seasons.

In [15]:
result = df[['id', 'text', 'name', 'episode_name', 'episode_number', 'season']]
# drop Random People
result = result[result['name'] != 'Random People']
# episode_name to upper case
result['episode_name'] = [x.upper() for x in result['episode_name']]

In [16]:
# add a 'episode - season' field to handle unique episodes 
ep_seas = []
for i, row in result.iterrows():
    ep_seas.append(str(row['season'])+'-'+str(row['episode_number']))

result['ep_seas'] = ep_seas
result.head()

,id,text,name,episode_name,episode_number,season,ep_seas
2,2,All right Jim. Your quarterlies look very goo...,Michael,PILOT,01,01,01-01
3,3,"Oh, I told you. I couldn't close it. So...",Jim,PILOT,01,01,01-01
4,4,So you've come to the master for guidance? Is...,Michael,PILOT,01,01,01-01
5,5,"Actually, you called me in here, but yeah.",Jim,PILOT,01,01,01-01
6,6,"All right. Well, let me show you how it's done.",Michael,PILOT,01,01,01-01


## *OPTIONAL - Save File

In [17]:
result.to_csv(root_path+'the_office_clean.csv', sep=';', encoding='utf-16', index=False)

# Prepare Features

In [18]:
df = result.copy()
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53847 entries, 2 to 60136
Data columns (total 7 columns):
id                53847 non-null int64
text              53847 non-null object
name              53847 non-null object
episode_name      53847 non-null object
episode_number    53847 non-null object
season            53847 non-null object
ep_seas           53847 non-null object
dtypes: int64(1), object(6)
memory usage: 3.3+ MB


## Define functions

Two functions will be used to create a second version of the text, a cleaner version without double spaces, punctuations, special characters and stop-words

*According to [yourdictionary.com](https://www.yourdictionary.com/stop-word), a stop-word definition is:  
    
    Noun  
    (computing) a word, usually one of a series in a stop list, that is to be ignored by a search engine etc
    
In this case, those are words without much meaning. So they'll be ignored in some parts of the analysis.

In [19]:
def clean_dialog(s):
    # replace special characters with space, remove underlines 
    # replace blank spaces with single blank space
    clean = re.sub('[^\w\s]', '', s)
    clean = re.sub('_', '', clean)
    clean = re.sub('\s+', ' ', clean)
    clean = clean.lower()
    return clean

stop_words = set(stopwords.words('english'))
stop_words_ext = ['na', 'da', 'dot', 'doo', 'a-wimowheh', 'parum', 'pum']

def remove_stopwords(words):
    i = []
    for word in words: 
        if word not in stop_words and word not in stop_words_ext: 
            i.append(word) 
    return ' '.join(i)

## Tokens

The clean texts are then tokenized into both words and sentences, and their respective tokens are counted and stored.

In [20]:
sent_token, sent_qty, word_token, word_qty, clean_txt = [],[],[],[],[]

for i, row in df.iterrows():
    # build tokens
    tokenized_text = sent_tokenize(str(row.text))
    tokenized_word = word_tokenize(clean_dialog(str(row.text)))
    # append to lists
    sent_token.append(tokenized_text)
    sent_qty.append(len(tokenized_text))
    word_token.append(tokenized_word)
    word_qty.append(len(tokenized_word))
    clean_txt.append(remove_stopwords(tokenized_word))

## Sentiment Analysis

The sentiment analysis was performed with VADER (Valence Aware Dictionary and sEntiment Reasoner)
https://github.com/cjhutto/vaderSentiment

VADER uses a dictionary to assign scores to the words, besides that it uses the position of the words and punctuation to score the document with the proportion of each sentiment contained on it. Those sentiments are named negative, positive and neutral.

After getting the proportions for each sentiment VADER also calculates a compound. The compound is basically a normalized sum of all proportions, goind from -1 (completely negative) to 1 (completely positive).

Some of the semantic contexts considered by VADER are:

    Conjunctions - E.g.: 'I like your X, but your Y is very bad';
    Negation Flips - E.g: 'This is not really the greatest';
    Degrees - E.g: 'This is good' vs 'This is extremely good';
    Capitalization - E.g: 'this is GREAT' vs 'this is great';
    Punctuation - E.g: 'this is great!!!' vs 'this is great';

In [21]:
# Sentiment analysis with Vader
analyser = SentimentIntensityAnalyzer()

neg, neu, pos, com = [], [], [], []

for i, row in df.iterrows():
    # generate polarity score
    score = analyser.polarity_scores(str(row.text))
    # append to lists
    neg.append(score.get('neg'))
    neu.append(score.get('neu'))
    pos.append(score.get('pos'))
    com.append(score.get('compound'))

## Add data

The dataframe is extended with the fields:
    
    clean text;
    sentences (tokens);
    words (tokens);
    sentences quantity (amount of sentences in a dialog);
    words quantity (amount of words in a dialog);
    negative (negative score for the dialog);
    positive (positive score for the dialog);
    neutral (neutral score for the dialog);
    compound (overall compound score for the dialog);

In [22]:
# add new features to dataframe
df['clean_txt'] = clean_txt
df['sentences'] = sent_token
df['words'] = word_token
df['sentences_qty'] = sent_qty
df['words_qty'] = word_qty
# sentiment analysis fields
df['negative']=neg
df['neutral']=neu
df['positive']=pos
df['compound']=com

df.head()

,id,text,name,episode_name,episode_number,season,ep_seas,clean_txt,sentences,words,sentences_qty,words_qty,negative,neutral,positive,compound
2,2,All right Jim. Your quarterlies look very goo...,Michael,PILOT,01,01,01-01,right jim quarterlies look good things library,"[ All right Jim., Your quarterlies look very g...","[all, right, jim, your, quarterlies, look, ver...",3,14,0.0,0.803,0.197,0.4927
3,3,"Oh, I told you. I couldn't close it. So...",Jim,PILOT,01,01,01-01,oh told couldnt close,"[ Oh, I told you., I couldn't close it., So...]","[oh, i, told, you, i, couldnt, close, it, so]",3,9,0.0,1.000,0.000,0.0000
4,4,So you've come to the master for guidance? Is...,Michael,PILOT,01,01,01-01,youve come master guidance youre saying grassh...,"[ So you've come to the master for guidance?, ...","[so, youve, come, to, the, master, for, guidan...",2,14,0.0,1.000,0.000,0.0000
5,5,"Actually, you called me in here, but yeah.",Jim,PILOT,01,01,01-01,actually called yeah,"[ Actually, you called me in here, but yeah.]","[actually, you, called, me, in, here, but, yeah]",1,8,0.0,0.714,0.286,0.4215
6,6,"All right. Well, let me show you how it's done.",Michael,PILOT,01,01,01-01,right well let show done,"[ All right., Well, let me show you how it's d...","[all, right, well, let, me, show, you, how, it...",2,10,0.0,0.811,0.189,0.2732


## Save File

In [23]:
df.to_csv(root_path+'the_office_features.csv', sep=';', encoding='utf-16', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53847 entries, 2 to 60136
Data columns (total 16 columns):
id                53847 non-null int64
text              53847 non-null object
name              53847 non-null object
episode_name      53847 non-null object
episode_number    53847 non-null object
season            53847 non-null object
ep_seas           53847 non-null object
clean_txt         53847 non-null object
sentences         53847 non-null object
words             53847 non-null object
sentences_qty     53847 non-null int64
words_qty         53847 non-null int64
negative          53847 non-null float64
neutral           53847 non-null float64
positive          53847 non-null float64
compound          53847 non-null float64
dtypes: float64(4), int64(3), object(9)
memory usage: 7.0+ MB
